This notebook can be used to run the simulation instead of having to type the commands into the command line interface

In [ ]:
# Copyright (c) 2022 The MATCH Authors. All rights reserved.
# Licensed under the GNU AFFERO GENERAL PUBLIC LICENSE Version 3 (or later), which is in the LICENSE file.

import multiprocessing as mp
import os
from pathlib import Path
import match_model.generate_input_files as generate_input_files
import shutil
import time
from nb_utilities import refresh_folder

# Enter the path to the folder where your model inputs are located
Specify the location of the model runs folder you will be using, either as a relative filepath or an absolute filepath. The default location is the MODEL_RUNS directory within the Git repo


In [ ]:
########################################################################
model_run_location = '../../MODEL_RUNS/example'
########################################################################

# ensure that the filepath string uses forward slashes rather than backslashes
model_run_location = model_run_location.replace("\\", "/")

# if the location is a relative filepath
if "../" in model_run_location:
    model_workspace = Path.cwd() / model_run_location
# otherwise if it is an absoluate filepaht
elif ":/" in model_run_location:
    model_workspace = Path(model_run_location)

# Generate and Validate model inputs

The following cell will load all of the input data from the `model_inputs.xlsx` file in your run folder, validate the inputs, and create all of the required input files in the format required by the solver. 

This cell will perform the following steps:  
1. If input files do not yet exist, create them
2. If input files exist, check that the current software version matches the version that was used to generate the inputs
3. If the versions do not match, re-generate the input files to ensure consistency

In [ ]:
# check if the directory exists
if os.path.exists(model_workspace / "inputs"):
    # check that an inputs version file exists
    # list of files and folders to keep
    keep_files = ['set_1','set_2','set_3', 'set_4', 'cbc', 'cbc.exe', 'model_inputs.xlsx']
    refresh_folder(model_workspace, keep_files=keep_files)

    if os.path.exists(model_workspace / "inputs_version.txt"):
        # get the version number
        with open(model_workspace / "inputs_version.txt", "r") as i:
            inputs_version = i.read()
        # get the current version number
        version_path = Path.cwd() / "version.py"
        version = {}
        with open(version_path) as f:
            exec(f.read(), version)
        version = version["__version__"]
        # check if the versions match
        if version == inputs_version:
            print(
                f"Input files already generated with current software version ({version})"
            )
        else:
            print(
                f"Inputs were generated using version {inputs_version}. Current version is {version}"
            )
            print(
                "Removing old files and re-generating inputs with current version. Please wait..."
            )
            old_files = os.listdir(model_workspace)
            old_files.remove("model_inputs.xlsx")
            for f in old_files:
                try:
                    os.remove(model_workspace / f)
                except PermissionError:
                    shutil.rmtree(model_workspace / f)
            generate_input_files.generate_inputs(model_workspace)
    else:
        generate_input_files.generate_inputs(model_workspace)
# if the directory does not exist, generate the input files
else:
    print("Generating inputs now...")
    generate_input_files.generate_inputs(model_workspace)

# Solve Model Scenarios

## Windows Operating Systems:

In [ ]:
os.system(
    f'start cmd /k "cd {model_run_location} & activate match_model & match solve-scenarios"'
)

## Running with Mac / Linux:

Steps to run on mac:
1. Open a new Terminal, and navigate to the folder with the homework/model_inputs.xslx.
2. Run `mamba activate match_model` 
3. run `match solve-scenarios`

If not you can open the summary_reports/ ...._.html file in a browser and investigate the results of your model run.



# Helpers for Rerunning Model
After you have run one scenario if you would like to rerun the scenario you will need to delete certain files.

To regenerate the input dataset:

In [ ]:
# list of files and folders to keep
keep_files = ['set_1', 'cbc', 'model_inputs.xlsx']
refresh_folder(model_workspace, keep_files=keep_files)

To rerun the match_model solver.... ie to run "match solve-scenarios":

In [ ]:
# list of files and folders to keep
keep_files = ['cbc' , 'inputs_version.txt' , 'options.txt', 'set_1', 'inputs', 'model_inputs.xlsx', 'outputs', 'scenarios.txt', 'summary_reports' ]
refresh_folder(model_workspace, keep_files=keep_files)